# TIN Interpolation from points
- This script creates a vector TIN multipolygon out of the points

In [2]:
import geopandas as gpd
import numpy as np
from scipy.spatial import Delaunay
from shapely.geometry import Polygon, box
import os
import pandas as pd

In [2]:
"""Export a certain area of the shapefile"""
# Load the shapefile
shapefile_path = r"C:\Users\ruben.crespo\Documents\03_tests\Res properties layer-KEN\Kenya_building shapefile\buildings_4326.shp"
gdf = gpd.read_file(shapefile_path)

# Define the bounding box of the area of interest (xmin, ymin, xmax, ymax)
xmin, ymin, xmax, ymax = 36.508395485, -1.475357495, 37.273437225, -1.081326365
# 36.4341,-1.4812 : 37.1991,-1.0871
area_of_interest = box(xmin, ymin, xmax, ymax)

# Filter points within the area of interest using spatial indexing
gdf_filtered = gdf[gdf.geometry.within(area_of_interest)]

# Export the filtered points to a new shapefile
output_filtered_points_path = r"C:\Users\ruben.crespo\Documents\03_tests\Res properties layer-KEN\Kenya_building shapefile\filtered_points.shp"
gdf_filtered.to_file(output_filtered_points_path)

In [4]:
"""Create TIN"""
# I don't want to repeat the computation
output_filtered_points_path = r"C:\Users\ruben.crespo\Documents\03_tests\Res properties layer-KEN\Kenya_building shapefile\filtered_points.shp"
gdf_filtered = gpd.read_file(output_filtered_points_path)

# Extract the coordinates of the filtered points
points = np.array([[point.x, point.y] for point in gdf_filtered.geometry])

# Perform Delaunay triangulation
tri = Delaunay(points)

# Optionally, save the TIN as a new shapefile (triangles)
triangles = []
for simplex in tri.simplices:
    triangle = Polygon(points[simplex])
    triangles.append(triangle)

# Create a GeoDataFrame with the triangles
tin_gdf = gpd.GeoDataFrame({'geometry': triangles}, crs=gdf.crs)

tin_gdf.to_file(r"C:\Users\ruben.crespo\Documents\03_tests\Res properties layer-KEN\Kenya_building shapefile\tin.shp")